In [8]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import requests_cache
import time
import math
from sqlalchemy import create_engine
import execjs
import random
import json

%matplotlib inline

engine = create_engine('sqlite:///stock.sqlite')
conn = engine.connect()

import re
def loads_jsonp(_jsonp):
    try:
        return json.loads(re.match(".*?({.*}).*",_jsonp,re.S).group(1))
    except:
        raise ValueError('Invalid Input')

def call_sse_api(data={}):
    headers = {'X-Requested-With': 'XMLHttpRequest',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) ' 'Chrome/56.0.2924.87 Safari/537.36',
               'Referer': 'http://www.sse.com.cn/assortment/stock/list/share/'
              }
    sse_stock_list_url = 'http://query.sse.com.cn/commonQuery.do'
    data['jsonCallBack'] = 'jsonpCallback' + str(math.floor(1e5 * random.random()))
    data['_'] = str(int(round(time.time() * 1000)))
    
    r = requests.post(sse_stock_list_url, data=data, headers=headers)
    return loads_jsonp(r.text)

In [9]:
%%time
stock_list = pd.read_sql_table('stock_list', conn, index_col='index')
df = pd.DataFrame()
for code in stock_list['代码']:
    data = {
        'isPagination': 'false',
        'sqlId': 'COMMON_SSE_ZQPZ_GP_GPLB_C',
        'productid': str(code),
    }
    df = df.append(pd.DataFrame(call_sse_api(data)['result'][0], index=[0]), ignore_index=True)

CPU times: user 26.5 s, sys: 794 ms, total: 27.3 s
Wall time: 3min 10s


In [10]:
df

,STATE_CODE_B_DESC,COMPANY_ABBR,SCU_TYPE,AREA_NAME_DESC,OPERATION_SEQ,COMPANY_ADDRESS,LEGAL_REPRESENTATIVE,ISHLT,SECURITY_CODE_A_SZ,SECURITY_CODE_A,...,OTHER_ABBR,FULL_NAME_IN_ENGLISH,CSRC_MIDDLE_CODE_DESC,SEC_NAME_FULL,WWW_ADDRESS,SECURITY_ABBR_A,CSRC_CODE_DESC,CHANGEABLE_BOND_ABBR,OFFICE_ADDRESS,REPR_PHONE
0,-,浦发银行,-,上海,892a598dd70404555fd2a8a7e3f37866,上海市中山东一路12号,郑杨,-,-,600000,...,浦发转股,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",-,浦发银行,http://www.spdb.com.cn,浦发银行,金融业,浦发转债,上海市中山东一路12号,-
1,-,白云机场,-,广东,c83072a44ad72a8fe9ed601fb00aa896,广州白云国际机场南工作区自编一号,邱嘉臣,-,-,600004,...,白云转股,"Guangzhou Baiyun International Airport Co.,ltd.",-,白云机场,www.baiyunport.com,白云机场,交通运输、仓储和邮政业,-,广州白云国际机场南工作区机场股份公司本部办公楼,-
2,-,东风汽车,-,湖北,82181a39282a32780e3de2dc3bbedbd0,湖北省襄阳市高新区东风汽车大道劲风路3幢,丁绍斌,-,-,600006,...,-,"Dongfeng Automobile Co.,LTD",-,东风汽车,http://www.dfac.com,东风汽车,制造业,-,湖北省武汉经济技术开发区创业路58号,-
3,-,中国国贸,-,北京,15c495e578ebab23a0441a690810722f,北京市建国门外大街1号,林明志,-,-,600007,...,-,China World Trade Center Company Ltd.,-,中国国贸,http://www.cwtc.com,中国国贸,房地产业,-,北京市建国门外大街1号国贸大厦A座29层,-
4,-,首创股份,-,北京,a6d51fbdaeeee53927e14a6a6e5ce46f,北京市西城区车公庄大街21号39幢16层,刘永政,-,-,600008,...,-,"BEIJING CAPITAL CO.,LTD",-,首创股份,http://www.capitalwater.cn,首创股份,电力、热力、燃气及水生产和供应业,-,北京市西城区车公庄大街21号2号楼,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,-,皖仪科技,-,安徽,ecd234e358182dca3e4e4c92359dd224,合肥高新区文曲路8号,臧牧,-,-,688600,...,-,"AnHui Wanyi Science and Technology Co., Ltd.",-,皖仪科技,http://www.wayeal.com.cn,皖仪科技,制造业,-,合肥高新区文曲路8号,-
1802,-,中控技术,-,浙江,14b044c3bae95c41fe774463e7aaf059,浙江省杭州市滨江区六和路309号,褚敏,-,-,688777,...,-,"Zhejiang Supcon Technology Co., Ltd.",-,中控技术,http://www.supcontech.com,中控技术,信息传输、软件和信息技术服务业,-,浙江省杭州市滨江区六和路309号中控科技园证券部,-
1803,-,科思科技,-,广东,e4c285cff93c4597a0a969343e26e36e,深圳市南山区西丽街道高新北区朗山路7号航空电子工程研发大厦五楼,刘建德,-,-,688788,...,-,"ShenZhen Consys Science&Technology Co., Ltd.",-,科思科技,http://www.consys.com.cn/,科思科技,制造业,-,深圳市南山区西丽街道高新北区朗山路7号航空电子工程研发大厦五楼,-
1804,-,中芯国际,-,开曼群岛,eb9bb33c22d88abcf1bcb1829c691e32,开曼群岛，KY1-1111，大开曼岛，哈钦斯道，板球广场,周子学,-,-,688981,...,-,Semiconductor Manufacturing International Corp...,-,中芯国际,http://www.smics.com/,中芯国际,制造业,-,中国上海市浦东新区张江路18号,-


In [3]:
df.to_sql('stock_base_data', engine, chunksize=1000, if_exists='replace')